## ISAAA crop genomics webscrape

**Description**: 

**Usage**: 

**Notes**: 

Last Edited: 

Author: LNC

Contact: lisa.cao@cybera.ca

In [1]:
#import libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import csv 
import lxml
from textblob import TextBlob
import regex as re

In [2]:
# get raw html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
soup = BeautifulSoup(gmo.content, "lxml")
# get text 
souptext = str(soup.get_text())

In [ ]:
# get rid of leading and trailing whitespace
genetext_raw = souptext.splitlines() 

# remove leading and trailing whitespace
genetext = []
for element in genetext_raw: 
    genetext_entry = element.strip()
    genetext.append(genetext_entry)

# remove empty list entries 
while('' in genetext) : 
    genetext.remove('') 

# slice out only parts we want
genetext = genetext[34:-15]
print(genetext)

In [ ]:
#generate list of traits
traits = [info for info in genetext if "Trait:" in info]
traits

In [ ]:
#generate list of gene names
genenames = []
for i in range(1, len(genetext)): 
    if genetext[i] == 'Source:': 
        genenames.append(genetext[i-1])
genenames

In [ ]:
#split list into sublists based on trait
sublist = []
traitinfo = []
for info in genetext: 
    if info in traits: 
        if sublist:
            traitinfo.append(sublist)
        sublist = [info]
    else: 
        sublist.append(info)
traitinfo.append(sublist)
print(traitinfo)

In [ ]:
#split sublist into sublists based on gene
gene = []
geneinfo = []
for sublist in traitinfo: 
    for info in sublist:
        if info in genenames: 
            if gene:
                geneinfo.append(gene)
            gene = [info]
        else: 
            gene.append(info)
    geneinfo.append(gene)
    print(geneinfo)

In [ ]:
#convert into dict? 


In [ ]:
#generate dataframe

In [ ]:
#regex way - not completed

In [3]:
# using blobtext
souptext = str(soup.get_text())
blobtext = TextBlob(souptext)
blobtext2 = str(blobtext)
print(blobtext2, file=open("blobtext.txt", "a")) # write to file

In [4]:
# dictionary for regex matching
gene_dict = {
    'trait': re.compile(r'Trait: (?P<trait>.*)\n'),
    'gene' : re.compile(r'\n(?P<gene>.*\s)\n Source:'),
    'source': re.compile(r'Source:\n(?P<source>.*)\n'), 
    'product': re.compile(r'Product:\n(?P<product>.*)\n'),
    'function': re.compile(r'Function:\n(?P<function>.*)\n')
}

In [7]:
def parseinfo(text):
    for info, regex in gene_dict.items():
            match = regex.search(text)
            if match: 
                return info, match
    return None, None